In [1]:
!pip install SpeechRecognition pyttsx3 pyaudio pandas scikit-learn


  Using cached PyAudio-0.2.14-cp311-cp311-win_amd64.whl.metadata (2.7 kB)
Using cached PyAudio-0.2.14-cp311-cp311-win_amd64.whl (164 kB)


In [5]:
import speech_recognition as sr
import pyttsx3
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Text-to-Speech setup
engine = pyttsx3.init()
def speak(text):
    print("Bot:", text)
    engine.say(text)
    engine.runAndWait()

# Load MovieLens movies dataset
df = pd.read_csv(r"C:\Users\Siya\.kaggle\kaggle.json\BollywoodMovieDetail.csv")  # adjust path if needed
df['description'] = df['genre'].str.replace('|', ' ')  # use genres as a basic description
df = df[['title', 'description']]
df['description'] = df['description'].fillna('')

# TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['description'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Recommendation Function
def get_recommendation(title):
    try:
        idx = df[df['title'].str.lower().str.contains(title.lower())].index[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:6]
        movie_indices = [i[0] for i in sim_scores]
        return df['title'].iloc[movie_indices].tolist()
    except:
        return []

# Voice Input
def get_voice_input():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio)
            print("You said:", text)
            return text
        except sr.UnknownValueError:
            speak("Sorry, I couldn't understand. Try again.")
            return None
        except sr.RequestError:
            speak("Could not request results; check your internet connection.")
            return None

# Main Chatbot Loop
def chatbot():
    speak("Hi! Tell me a movie you like, and I’ll recommend something similar.")
    while True:
        user_input = get_voice_input()
        if user_input:
            recommendations = get_recommendation(user_input)
            if recommendations:
                speak(f"If you liked {user_input}, you might also like: " + ", ".join(recommendations))
            else:
                speak("Sorry, I don't know that movie. Try another one.")
        while True:
            speak("Do you want another recommendation? Please say yes or no.")
            response = get_voice_input()
            
            if response:
                response = response.lower().strip()
                print("Recognized response:", response)

                if any(word in response for word in ['no', 'nope', 'nah', 'not really']):
                    speak("Okay, goodbye!")
                    return  # end the chatbot loop
                elif any(word in response for word in ['yes', 'yeah', 'yup', 'sure']):
                    break  # go back to outer loop to ask for another movie
                else:
                    speak("I heard: " + response + ". Please say yes or no.")
            else:
                speak("Didn't get that. Please say yes or no.")


# Run it!
chatbot()


C:\Users\Siya\AppData\Local\Temp\ipykernel_15080\114863290.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['description'] = df['genre'].str.replace('|', ' ')  # use genres as a basic description


Bot: Hi! Tell me a movie you like, and I’ll recommend something similar.
Listening...
You said: Devdas
Bot: If you liked Devdas, you might also like: Censor, Haan Maine Bhi Pyaar Kiya, Hum Pyar Tumhi Se Kar Baithe, Tumsa Nahin Dekha, Jaan-E-Mann: Let's Fall in Love... Again
Bot: Do you want another recommendation? Please say yes or no.
Listening...
You said: no thanks
Recognized response: no thanks
Bot: Okay, goodbye!
